In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
import scipy.linalg as sla

In [ ]:
from helper_functions import *

<div class="alert alert-info">
    
<h1>Example 2: Quantum Chemistry</h1>
    

In this example, we will be solving a generalized eigenvalue problem which has the form
$$
{\bf A x} = \lambda {\bf B x},
$$
where ${\bf A}$ and ${\bf B}$ are matrices.


This generalized eigenvalue problem arises in computational chemistry, when modeling the electron and nucleus interactions of molecules. The electronic structure of a molecule is governed by Schrödinger's equation, which can be approximated using the Hartree–Fock method:

$$
{\bf F \,C}_i = \lambda_i \,{\bf S\, C}_i \quad \text{(Roothaan's equations)}
$$

where ${\bf F}$ is the Fock matrix including the electron interactions,
${\bf C}$ is a matrix with the molecular coefficients for the atomic orbitals, $\lambda_i$ are the molecular energies for each atomic orbital, and ${\bf S}$ is a matrix with the basis functions used to discretize the problem.
    
Note that the eigenvectors are the columns ${\bf C}_i$ corresponding to the eigenvalues $\lambda_i$.
    
[Wikipedia reference](https://en.wikipedia.org/wiki/Roothaan_equations#:~:text=The%20Roothaan%20equations%20are%20a,%2C%20respectively%2C%20are%20doubly%20occupied)
    
</div>

<div class="alert alert-warning">    
    
#### One small issue...


The Fock matrix ${\bf F}$ depends on the ${\bf C}$ matrix! We have a **"chicken or the egg" paradox**, because we need to have  ${\bf F}$ to obtain ${\bf C}$, but ${\bf C}$ is the solution of the generalized eigenvalue problem. 
    
You just found the first **NONLINEAR** example introduced in this class! To solve nonlinear problems, we will use **iterative methods**.

We will start with an initial guess:

$${\bf C}^{(0)} $$

and then use the iterative scheme (Roothaan's equations):


$$
{\bf F}({\bf C}^{(n)}) \, {\bf C}^{(n+1)}_i = \lambda^{(n+1)}_i \,{\bf S\, C}^{(n+1)}_i 
$$
</div>

where we provide you with a function to evaluate the Fock matrix. 

```python
F = compute_Fock(C)
```

We will assume we are solving a problem where the matrices have shape `(7,7)` which corresponds to the H2O molecule. The matrix ${\bf S}$ is given below:

In [ ]:
# Load S matrix from file
n = 7
S =  np.load('S.npy')

For the first step of your iterative process, obtain the Fock matrix assuming ${\bf C}^{(0)} $ is a zero matrix. Store your result in the variable `F`

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
C=np.zeros((7,7))
F=compute_Fock(C)
print(F.shape)

We will first use the function [scipy.linalg.eigh](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html#scipy.linalg.eigh) to obtain the solution of the generalized eigenvalue problem for the first time step

$$
{\bf F}({\bf C}^{(0)}) \, {\bf C}^{(1)}_i = \lambda^{(1)}_i \,{\bf S\, C}^{(1)}_i
$$

Store the eigenvalues $\lambda^{(1)}_i$ in the 1d numpy array `lambn` and the eigenvectors ${\bf C}^{(1)}_i$ in the 2d numpy array `Cn`.

Note the grader is just checking the shape and type of `lambn` and `Cn`, but we will check the values in later tests.

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 

lambn, Cn = sla.eigh(F, S)

Check the values of the eigenvalues. Note that they are given in ascending order: 

In [ ]:
lambn

Compute the 2-norm of the eigenvalue array `lambn`. We will use this in order to check convergence of our iterative method. Store this as `ln`.

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
ln=la.norm(lambn,2)

Write a code snippet that implement the iterative method proposed above:

- Start from initial guess ${\bf C^{(0)} = 0}$

- Perform the computation below until convergence:

    - Compute the Fock matrix
    - Solve the generalized eigenvalue problem
    - Compute the norm of the eigenvalue array
    - Take the difference between the norm of two consecutive iterations. If the absolute value of this difference is below the tolerance, stop the iterations
    
- Use `tol = 1e-6`

**Define:**
At the end of your iterative method, the code snippet defined in the `#grade` cell below should define:

- The converged matrix eigenvector array, in this example the ${\bf C}$ matrix, stored in the variable `C_converged`

- The converged eigenvalue array stored in the variable `lambda_converged`
    

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
tol = 1e-6
C = np.zeros((n,n))
prev_ln = 0
curr_Cn=C
while True:
    Fn = compute_Fock(curr_Cn)
    curr_lambn, curr_Cn = sla.eigh(Fn, S)
    curr_ln=la.norm(curr_lambn,2)
    if np.abs(prev_ln - curr_ln) < tol:
        break
    prev_ln = curr_ln
    
C_converged = curr_Cn
lambda_converged = curr_lambn


In [ ]:
lambda_converged

array([-20.26205131,  -1.26227562,  -0.56937852,  -0.48077939,
        -0.39749241,   0.58252526,   0.65807393])

In [ ]:
C_converged
Fn = compute_Fock(C_converged)
ltt, Ctt = sla.eigh(Fn,S)

<div class="alert alert-warning">   

<b> An alternative solution </b>

<br>
Suppose you only have available a function that solves the eigenvalue problem

$$
{\bf A x} = \lambda {\bf x},
$$

Transform the original generalized eigenvalue problem 


$$
{\bf F}({\bf C}^{(n)}) \, {\bf C}^{(n+1)}_i = \lambda^{(n+1)}_i \,{\bf S\, C}^{(n+1)}_i 
$$


such that you can solve it using [numpy.linalg.eigh](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html#numpy.linalg.eigh)
    
</div>

We can first diagonalize the matrix ${\bf S}$, such that we write ${\bf S} = {\bf U} {\bf D}{\bf U}^T$. Compute the matrices ${\bf U}$ and ${\bf D}$ and store them in the variables `U` and `D` respectively. Both variables should be 2d numpy arrays.

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 


Note the grader is just checking the shape and type of `U` and `D`. If your implementation is correct, the expression below should evaluate as `True`

In [ ]:
np.allclose(U @ D @U.T , S)

Suppose we have a orthogonal matrix ${\bf X}$ such that ${\bf X}{\bf X}^T = {\bf X}^T{\bf X} = {\bf I}$. We can re-write the Roothaan's equations as:


$$
\color{red}{{\bf X}^T}{\bf F}({\bf C}^{(n)}) \,\color{blue}{{\bf X}{\bf X}^T}\, {\bf C}^{(n+1)}_i = \lambda^{(n+1)}_i \,\color{red}{{\bf X}^T}\,{\bf S}\,\color{blue}{{\bf X}{\bf X}^T}\,{ \bf C}^{(n+1)}_i 
$$

If we define the new variables:

$${\bf\bar{c}}_i = {\bf X}^T\, {\bf C}^{(n+1)}_i $$

$${\bf\bar{F}}_i = {\bf X}^T{\bf F}({\bf C}^{(n)}) \,{\bf X}$$

$${\bf\bar{S}}_i = {\bf X}^T{\bf S} \,{\bf X}$$

the Roothaan's equations become:

$$ {\bf\bar{F}}_i {\bf\bar{c}}_i = \lambda^{(n+1)}_i {\bf\bar{S}}_i {\bf\bar{c}}_i $$

If we create the matrix ${\bf X}$ such that:

$${\bf X} = {\bf U}{\bf D}^{-1/2}{\bf U}^T $$

where ${\bf U}$ and ${\bf D}$ come from the diagonalization of ${\bf S}$, one can show that 

$${\bf\bar{S}}_i = {\bf X}^T{\bf S} \,{\bf X} =  {\bf I}$$

<div class="alert alert-warning">

finaly resulting in the alternative iterative scheme:

$$ {\bf\bar{F}}_i {\bf\bar{c}}_i = \lambda^{(n+1)}_i  {\bf\bar{c}}_i $$
    
</div>

Write the function `solve_roothaan` that implements the above iterative method, which can be summarized as:

- Compute diagonalization of ${\bf S}$ (i.e.,obtain the matrices ${\bf U}$ and ${\bf D}$)

- Compute the matrix ${\bf X} =  {\bf U}{\bf D}^{-1/2}{\bf U}^T $

- Start from initial guess ${\bf C^{(0)} = 0}$

- Perform the computation below until convergence:

    - Compute the Fock matrix
    - Compute the modified Fock matrix ${\bf\bar{F}} = {\bf X}^T{\bf F} \,{\bf X}$
    - Solve the eigenvalue problem above using `numpy.linalg.eigh` to obtain the eigenpair $(\lambda^{(n+1)}_i, {\bf\bar{c}}_i)$
    - Evaluate the matrix  ${\bf C}^{(n+1)}$. Make sure you take into account that the matrix ${\bf X}$ is orthogonal, and hence ${\bf X}^{-1} = {\bf X}^T$.
    - Compute the norm of the eigenvalue array
    - Take the difference between the norm of two consecutive iterations. If the absolute value of this difference is below the tolerance, stop the iterations

- At the end of your iterative method, the function should return:

    - The converged eigenvectors, here the matrix ${\bf C}^{(n+1)}$, stored in the variable `C_converged`

    - The converged eigenvalue array stored in the variable `lambda_converged`

If you run your function for the matrix ${\bf S}$ provided in the first approach, you should get the same results. However, make sure you are not hard-coding this information, since the grading function can use different matrices and tolerance values.
    

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
def solve_roothaan(S,tol):
    
    C = np.zeros((n,n))
    
    C_converged = ...
    lambda_converged = ...
    

    return C_converged, lambda_converged




In [ ]:
# Call the function and check if you obtained the same results
C_converged_2, lambda_converged_2 = solve_roothaan(S,1e-6)

In [ ]:
C_converged_2, lambda_converged_2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
import scipy.linalg as sla

In [ ]:
from helper_functions import *

<div class="alert alert-info">
    
<h1>Example 2: Quantum Chemistry</h1>
    

In this example, we will be solving a generalized eigenvalue problem which has the form
$$
{\bf A x} = \lambda {\bf B x},
$$
where ${\bf A}$ and ${\bf B}$ are matrices.


This generalized eigenvalue problem arises in computational chemistry, when modeling the electron and nucleus interactions of molecules. The electronic structure of a molecule is governed by Schrödinger's equation, which can be approximated using the Hartree–Fock method:

$$
{\bf F \,C}_i = \lambda_i \,{\bf S\, C}_i \quad \text{(Roothaan's equations)}
$$

where ${\bf F}$ is the Fock matrix including the electron interactions,
${\bf C}$ is a matrix with the molecular coefficients for the atomic orbitals, $\lambda_i$ are the molecular energies for each atomic orbital, and ${\bf S}$ is a matrix with the basis functions used to discretize the problem.
    
Note that the eigenvectors are the columns ${\bf C}_i$ corresponding to the eigenvalues $\lambda_i$.
    
[Wikipedia reference](https://en.wikipedia.org/wiki/Roothaan_equations#:~:text=The%20Roothaan%20equations%20are%20a,%2C%20respectively%2C%20are%20doubly%20occupied)
    
</div>

<div class="alert alert-warning">    
    
#### One small issue...


The Fock matrix ${\bf F}$ depends on the ${\bf C}$ matrix! We have a **"chicken or the egg" paradox**, because we need to have  ${\bf F}$ to obtain ${\bf C}$, but ${\bf C}$ is the solution of the generalized eigenvalue problem. 
    
You just found the first **NONLINEAR** example introduced in this class! To solve nonlinear problems, we will use **iterative methods**.

We will start with an initial guess:

$${\bf C}^{(0)} $$

and then use the iterative scheme (Roothaan's equations):


$$
{\bf F}({\bf C}^{(n)}) \, {\bf C}^{(n+1)}_i = \lambda^{(n+1)}_i \,{\bf S\, C}^{(n+1)}_i 
$$
</div>

where we provide you with a function to evaluate the Fock matrix. 

```python
F = compute_Fock(C)
```

We will assume we are solving a problem where the matrices have shape `(7,7)` which corresponds to the H2O molecule. The matrix ${\bf S}$ is given below:

In [ ]:
# Load S matrix from file
n = 7
S =  np.load('S.npy')

For the first step of your iterative process, obtain the Fock matrix assuming ${\bf C}^{(0)} $ is a zero matrix. Store your result in the variable `F`

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
C=np.zeros((7,7))
F=compute_Fock(C)
print(F.shape)

We will first use the function [scipy.linalg.eigh](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html#scipy.linalg.eigh) to obtain the solution of the generalized eigenvalue problem for the first time step

$$
{\bf F}({\bf C}^{(0)}) \, {\bf C}^{(1)}_i = \lambda^{(1)}_i \,{\bf S\, C}^{(1)}_i
$$

Store the eigenvalues $\lambda^{(1)}_i$ in the 1d numpy array `lambn` and the eigenvectors ${\bf C}^{(1)}_i$ in the 2d numpy array `Cn`.

Note the grader is just checking the shape and type of `lambn` and `Cn`, but we will check the values in later tests.

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 

lambn, Cn = sla.eigh(F, S)

Check the values of the eigenvalues. Note that they are given in ascending order: 

In [ ]:
lambn

Compute the 2-norm of the eigenvalue array `lambn`. We will use this in order to check convergence of our iterative method. Store this as `ln`.

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
ln=la.norm(lambn,2)

Write a code snippet that implement the iterative method proposed above:

- Start from initial guess ${\bf C^{(0)} = 0}$

- Perform the computation below until convergence:

    - Compute the Fock matrix
    - Solve the generalized eigenvalue problem
    - Compute the norm of the eigenvalue array
    - Take the difference between the norm of two consecutive iterations. If the absolute value of this difference is below the tolerance, stop the iterations
    
- Use `tol = 1e-6`

**Define:**
At the end of your iterative method, the code snippet defined in the `#grade` cell below should define:

- The converged matrix eigenvector array, in this example the ${\bf C}$ matrix, stored in the variable `C_converged`

- The converged eigenvalue array stored in the variable `lambda_converged`
    

In [54]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
tol = 1e-6
C = np.zeros((n,n))
prev_ln = 0
curr_Cn=C
while True:
    Fn = compute_Fock(curr_Cn)
    curr_lambn, curr_Cn = sla.eigh(Fn, S)
    curr_ln=la.norm(curr_lambn,2)
    if np.abs(prev_ln - curr_ln) < tol:
        break
    prev_ln = curr_ln
    
C_converged = curr_Cn
lambda_converged = curr_lambn


In [55]:
lambda_converged

array([-20.26205131,  -1.26227562,  -0.56937852,  -0.48077939,
        -0.39749241,   0.58252526,   0.65807393])

In [43]:
C_converged
Fn = compute_Fock(C_converged)
ltt, Ctt = sla.eigh(Fn,S)

<div class="alert alert-warning">   

<b> An alternative solution </b>

<br>
Suppose you only have available a function that solves the eigenvalue problem

$$
{\bf A x} = \lambda {\bf x},
$$

Transform the original generalized eigenvalue problem 


$$
{\bf F}({\bf C}^{(n)}) \, {\bf C}^{(n+1)}_i = \lambda^{(n+1)}_i \,{\bf S\, C}^{(n+1)}_i 
$$


such that you can solve it using [numpy.linalg.eigh](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html#numpy.linalg.eigh)
    
</div>

We can first diagonalize the matrix ${\bf S}$, such that we write ${\bf S} = {\bf U} {\bf D}{\bf U}^T$. Compute the matrices ${\bf U}$ and ${\bf D}$ and store them in the variables `U` and `D` respectively. Both variables should be 2d numpy arrays.

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 


Note the grader is just checking the shape and type of `U` and `D`. If your implementation is correct, the expression below should evaluate as `True`

In [ ]:
np.allclose(U @ D @U.T , S)

Suppose we have a orthogonal matrix ${\bf X}$ such that ${\bf X}{\bf X}^T = {\bf X}^T{\bf X} = {\bf I}$. We can re-write the Roothaan's equations as:


$$
\color{red}{{\bf X}^T}{\bf F}({\bf C}^{(n)}) \,\color{blue}{{\bf X}{\bf X}^T}\, {\bf C}^{(n+1)}_i = \lambda^{(n+1)}_i \,\color{red}{{\bf X}^T}\,{\bf S}\,\color{blue}{{\bf X}{\bf X}^T}\,{ \bf C}^{(n+1)}_i 
$$

If we define the new variables:

$${\bf\bar{c}}_i = {\bf X}^T\, {\bf C}^{(n+1)}_i $$

$${\bf\bar{F}}_i = {\bf X}^T{\bf F}({\bf C}^{(n)}) \,{\bf X}$$

$${\bf\bar{S}}_i = {\bf X}^T{\bf S} \,{\bf X}$$

the Roothaan's equations become:

$$ {\bf\bar{F}}_i {\bf\bar{c}}_i = \lambda^{(n+1)}_i {\bf\bar{S}}_i {\bf\bar{c}}_i $$

If we create the matrix ${\bf X}$ such that:

$${\bf X} = {\bf U}{\bf D}^{-1/2}{\bf U}^T $$

where ${\bf U}$ and ${\bf D}$ come from the diagonalization of ${\bf S}$, one can show that 

$${\bf\bar{S}}_i = {\bf X}^T{\bf S} \,{\bf X} =  {\bf I}$$

<div class="alert alert-warning">

finaly resulting in the alternative iterative scheme:

$$ {\bf\bar{F}}_i {\bf\bar{c}}_i = \lambda^{(n+1)}_i  {\bf\bar{c}}_i $$
    
</div>

Write the function `solve_roothaan` that implements the above iterative method, which can be summarized as:

- Compute diagonalization of ${\bf S}$ (i.e.,obtain the matrices ${\bf U}$ and ${\bf D}$)

- Compute the matrix ${\bf X} =  {\bf U}{\bf D}^{-1/2}{\bf U}^T $

- Start from initial guess ${\bf C^{(0)} = 0}$

- Perform the computation below until convergence:

    - Compute the Fock matrix
    - Compute the modified Fock matrix ${\bf\bar{F}} = {\bf X}^T{\bf F} \,{\bf X}$
    - Solve the eigenvalue problem above using `numpy.linalg.eigh` to obtain the eigenpair $(\lambda^{(n+1)}_i, {\bf\bar{c}}_i)$
    - Evaluate the matrix  ${\bf C}^{(n+1)}$. Make sure you take into account that the matrix ${\bf X}$ is orthogonal, and hence ${\bf X}^{-1} = {\bf X}^T$.
    - Compute the norm of the eigenvalue array
    - Take the difference between the norm of two consecutive iterations. If the absolute value of this difference is below the tolerance, stop the iterations

- At the end of your iterative method, the function should return:

    - The converged eigenvectors, here the matrix ${\bf C}^{(n+1)}$, stored in the variable `C_converged`

    - The converged eigenvalue array stored in the variable `lambda_converged`

If you run your function for the matrix ${\bf S}$ provided in the first approach, you should get the same results. However, make sure you are not hard-coding this information, since the grading function can use different matrices and tolerance values.
    

In [ ]:
#grade (enter your code in this cell - DO NOT DELETE THIS LINE) 
def solve_roothaan(S,tol):
    
    C = np.zeros((n,n))
    
    C_converged = ...
    lambda_converged = ...
    

    return C_converged, lambda_converged




In [ ]:
# Call the function and check if you obtained the same results
C_converged_2, lambda_converged_2 = solve_roothaan(S,1e-6)

In [ ]:
C_converged_2, lambda_converged_2